In [ ]:
import pandas as pd
from neo_finrl.env_fx_trading.env_fx_bs import tgym
import time
import datetime
import gym
from stable_baselines3 import PPO, TD3
from stable_baselines3.common.env_util import make_vec_env
from neo_finrl.env_fx_trading.util.action_enum import ActionEnum, form_action
file ="./data/split/GBPUSD/weekly/GBPUSD_2018_56.csv"
df = pd.read_csv(file)
env_name ="TradingGym-v1"
# t = gym.make(env_name)
t = tgym(df)
# if model: del model # remove to demonstrate saving and loading
model_name='ppo_fx-2017-5'

if model_name.startswith('ppo'): 
    model = PPO.load(model_name)
else:
    model = TD3.load(model_name)
start_time = time.time()
obs = t.reset()
t.current_step=0
done = False
while not done:
    action, _states = model.predict(obs)
    obs, rewards, done, info = t.step(action)
    # print(action[0])
t.render(mode='graph')
print(f"--- running time: {(time.time() - start_time)}---")

In [ ]:
from neo_finrl.env_fx_trading.util.plot_chart import TradingChart
p = TradingChart(t.df, t.transaction_history)
p.transaction_line('GBPUSD')

In [ ]:
import pandas as pd
from neo_finrl.env_fx_trading.env_fx import tgym
import time
import datetime
import gym
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
file ="./data/split/GBPUSD/weekly/GBPUSD_2017_2.csv"
df = pd.read_csv(file)
env_name ="TradingGym-v1"
env = gym.make(env_name)

In [ ]:
from neo_finrl.env_fx_trading.util.action_enum import ActionEnum, form_action
from stable_baselines3 import PPO
model = PPO.load("test")
obs = env.reset()
done=False
step = 0
while not done:
    action, _states = model.predict(obs)
    obs, rewards, done, info = env.step(action)
    step +=1
    a, r = form_action(action[0])
    print(step, action, a, r , rewards)
env.render(mode='graph')

# Market Order Sell first.
Only available one holding position. 

In [ ]:
import pandas as pd
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv,VecNormalize
from stable_baselines3 import PPO
from neo_finrl.env_fx_trading.env_fx_1 import TradingSellEnv
from neo_finrl.env_fx_trading.util.action_enum import ActionEnum, form_action
i = 0
file = f"./data/split/GBPUSD/weekly/GBPUSD_2017_{i}.csv"
df = pd.read_csv(file)
env = DummyVecEnv([lambda: TradingSellEnv(df=df)] )
# Automatically normalize the input features and reward
env_nor = VecNormalize(env, norm_obs=True, norm_reward=True, clip_obs=10.)

obs = env.reset()
print(obs)
obs = env_nor.reset()
print(obs)


In [ ]:
action = env_nor.action_space.sample()
obs, rewards, done, info = env.step(action)
print(obs)
obs, rewards, done, info = env_nor.step(action)
print(obs)

In [2]:
import os
import pandas as pd
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv,VecNormalize
from stable_baselines3 import PPO
from neo_finrl.env_fx_trading.env_fx_1 import TradingSellEnv
from neo_finrl.env_fx_trading.util.action_enum import ActionEnum, form_action
def train(df, total_timesteps,model_name, num_cpu = 4, **policy_kwargs):
    # num_cpu Number of processes to use
    # Create the vectorized environment
    # env = SubprocVecEnv([make_env(env_id, i) for i in range(num_cpu)])

    # Stable Baselines provides you with make_vec_env() helper
    # which does exactly the previous steps for you.
    # You can choose between `DummyVecEnv` (usually faster) and `SubprocVecEnv`
    # env = make_vec_env(t, n_envs=num_cpu, seed=0, vec_env_cls=DummyVecEnv)
    env = DummyVecEnv([lambda: TradingSellEnv(df=df) for _ in range(num_cpu)] )
    # Automatically normalize the input features and reward
    env = VecNormalize(env, norm_obs=True, norm_reward=True, clip_obs=10.)
    if os.path.exists(model_name + '.zip'):
        model = PPO.load(model_name, env=env, )
        print(f'retaining model: {model_name}')
        model.learn(total_timesteps=total_timesteps,**policy_kwargs)
        model.save(model_name)
        env.save("vec_normalize.pkl")
    else:
        print(f'new model: {model_name}')
        model = PPO('MlpPolicy', env=env, **policy_kwargs)
        model.learn(total_timesteps=total_timesteps)
        model.save(model_name)   

model_name = "ppo_fx-2017-7"   
for i in range(0,1): # one year 2017
    
    file = f"./data/split/GBPUSD/weekly/GBPUSD_2017_{i}.csv"
    if os.path.isfile(file):
        print(file)
        df = pd.read_csv(file)
        total_timesteps=20000
        train(df,total_timesteps,model_name)

./data/split/GBPUSD/weekly/GBPUSD_2017_0.csv
initial done:
observation_list:['Open', 'High', 'Low', 'Close', 'minute', 'hour', 'day', 'macd', 'boll_ub', 'boll_lb', 'rsi_30', 'dx_30', 'close_30_sma', 'close_60_sma']
 assets:['GBPUSD']
 time serial: 2017-01-02 01:00:00 -> 2017-01-06 23:55:00 length: 1428
initial done:
observation_list:['Open', 'High', 'Low', 'Close', 'minute', 'hour', 'day', 'macd', 'boll_ub', 'boll_lb', 'rsi_30', 'dx_30', 'close_30_sma', 'close_60_sma']
 assets:['GBPUSD']
 time serial: 2017-01-02 01:00:00 -> 2017-01-06 23:55:00 length: 1428
initial done:
observation_list:['Open', 'High', 'Low', 'Close', 'minute', 'hour', 'day', 'macd', 'boll_ub', 'boll_lb', 'rsi_30', 'dx_30', 'close_30_sma', 'close_60_sma']
 assets:['GBPUSD']
 time serial: 2017-01-02 01:00:00 -> 2017-01-06 23:55:00 length: 1428
initial done:
observation_list:['Open', 'High', 'Low', 'Close', 'minute', 'hour', 'day', 'macd', 'boll_ub', 'boll_lb', 'rsi_30', 'dx_30', 'close_30_sma', 'close_60_sma']
 assets:

TypeError: 'numpy.int64' object is not iterable